In [ ]:
!pip install pickle5

In [ ]:
import numpy as np
import pandas as pd
import time

import random

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
from tqdm import tqdm

import pickle5 as pickle
from scipy import stats

import matplotlib.ticker as tick

import sys
sys.path.append('..')
from nsgp_vi_3ns_params.py import nsgp_vi_3ns_params

# We'll use double precision throughout for better numerics.
dtype = np.float64

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

plt.style.use('ggplot') 
plt.style.use('seaborn-paper')
plt.style.use('seaborn-whitegrid')


In [ ]:
print(tf.__version__)
print(tfp.__version__)

In [ ]:
print(tfp.__version__)
print(tf.__version__)

In [ ]:
# data for the Sceaux household 

df = pd.read_csv('../data/household_power_consumption.csv',thousands=',')#sep=';'

#drop missing values
df = df.dropna()

dates = pd.DatetimeIndex(df['Date'],dayfirst=True)
times = pd.DatetimeIndex(df['Time'])

T = (24*(dates.day-1) + times.hour  + times.minute/60.).astype(float).values[:,None]
# time in hours !
X = df['Global_active_power'].astype('float').values.reshape(len(T),1)

X = np.log(X)

print(T.shape)
print(X.shape)

In [ ]:
# data for the city of Tetuan 

#df = pd.read_csv('../data/Tetuan City power consumption.csv', thousands=',')
#dts = pd.DatetimeIndex(df['DateTime'],dayfirst=True)
#T = (24*(dts.day-1) + dts.hour  +dts.minute/60.).astype(float).values[:,None]
# in hours !
#X = df['Zone 1 Power Consumption'].astype('float').values.reshape(len(T),1) # if Zone 1 is analysed

#X = df['Zone 3  Power Consumption'].astype('float').values.reshape(len(T),1) # if Zone 3 is analysed

# 3 zones

#X = np.log(X)

#print(T.shape)
#print(X.shape)

In [ ]:
# Only for section 6.3 of the paper; 

#Sceaux household
#T=T[:-200000]
#X=X[:-200000]

#Tetuan datasets
#T=T[:-10000]
#X=X[:-10000]

In [ ]:
print(len(T))
print(len(X))

In [ ]:
plt.plot(T,X,'.',markersize=5)
plt.xlabel('Time (hours)')
plt.ylabel('Std. obs. (Global active power in killowatts)')
plt.tick_params(axis='both', which='major', labelsize=10)
plt.savefig("Power_data_vi.pdf",dpi=600)
plt.show()

In [ ]:
#Produces plots in Fig. 2 of the paper

minutes = (60.*times.hour  + times.minute).astype(float).values
gap = X[:,0]

bin_means, bin_edges, binnumber = stats.binned_statistic(minutes, gap, statistic='mean', bins=48)
bin_stdv, bin_edges, binnumber = stats.binned_statistic(minutes, gap, statistic='std', bins=48)

bin_width = (bin_edges[1] - bin_edges[0])
bin_centers = bin_edges[1:] - bin_width/2

plt.figure()
plt.plot(bin_centers/60., bin_means,color='C1',label='Sceaux mean')
plt.fill_between(bin_centers/60.,bin_means- bin_stdv,bin_means + bin_stdv,color='C1',alpha=0.25)
plt.xlabel('Time (hours)',size=10)
plt.ylabel('Log power consumption per minute',size=10)
plt.legend(fontsize=10)
plt.text(-2.5,1.8,'A', size=20)#, transform=ax[1, 1].transAxes)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.savefig("Power_log_data_Sceaux.pdf",dpi=600)
plt.show()


In [ ]:
# Produces raw data plots in the supplementary materials

f, ax1 = plt.subplots(1, 1,figsize=(20,10))
ax1.hist(X)
ax1.set_xlabel('Number of observations',size=15)
ax1.set_ylabel('Power consumption observations',size=15)
ax1.text(-0.05,1,'A', size=20, transform=ax1.transAxes)
ax1.tick_params(axis='both', which='major', labelsize=15)
tick_locator = tick.MaxNLocator(nbins=7)
plt.savefig("Raw_power_Sceaux_data_hist.pdf",dpi=600)


In [ ]:
df

In [ ]:
num_training_points_ = T.shape[0]

num_inducing_points_ = 48

inducing_index_points = np.linspace(0., 24., num_inducing_points_, endpoint=False)[..., np.newaxis]
np.random.shuffle(inducing_index_points)


In [ ]:
BATCH_SIZE=8
SEG_LENGTH=500 


class segment_generator:
    def __iter__(self):
        
        # loop over segments
        self.j = 0
        self.max_j = num_training_points_//SEG_LENGTH
        
        
        return self

    def __next__(self):
        

        if self.j==self.max_j:
            raise StopIteration

        TT = T[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
        XX = X[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
    
        self.j += 1

        return TT,XX

        
dataset = tf.data.Dataset.from_generator(segment_generator, (tf.float64)) 
dataset = dataset.map(lambda dd: (dd[0],dd[1]))
dataset = dataset.shuffle(500)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
#for d in dataset:
   # print(d[0].shape,d[1].shape)

In [ ]:
#lengthscale kernel parameters,lower levels
kernel_len_a = tfp.util.TransformedVariable(0.4, tfb.Softplus(),dtype=tf.float64, name='k_len_a',trainable=True)
kernel_len_l = tfp.util.TransformedVariable(0.4, tfb.Chain([tfb.Shift(np.float64(0.0)), tfb.Softplus()]),dtype=tf.float64, name='k_len_l',trainable=True)

# amplitude kernel parameters, lower levels
kernel_amp_a = tfp.util.TransformedVariable(0.4, tfb.Softplus(),dtype=tf.float64, name='k_amp_a',trainable=True)
kernel_amp_l = tfp.util.TransformedVariable(0.4, tfb.Chain([tfb.Shift(np.float64(0.0)), tfb.Softplus()]),dtype=tf.float64, name='k_amp_l',trainable=True)

# mean kernel parameters, lower levels
kernel_mean_a = tfp.util.TransformedVariable(1.0, tfb.Softplus(),dtype=tf.float64, name='k_mean_a',trainable=True)
kernel_mean_l = tfp.util.TransformedVariable(1.0, tfb.Chain([tfb.Shift(np.float64(0.5)), tfb.Softplus()]),dtype=tf.float64, name='k_mean_l',trainable=True)

#kernels on the second layer
kernel_len = tfk.ExpSinSquared(kernel_len_a,kernel_len_l,period=np.float64(24.0))
kernel_amp = tfk.ExpSinSquared(kernel_amp_a,kernel_amp_l,period=np.float64(24.0))
kernel_mean = tfk.ExpSinSquared(kernel_mean_a,kernel_mean_l,period=np.float64(24.0))

#added kernel_mean
vgp = nsgp_vi_3ns_params(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  


In [ ]:
# code when the lengthscale and signal variance are constants, but only the mean is modelled by a GP

#kernel_mean_a = tfp.util.TransformedVariable(1.0, tfb.Softplus(),dtype=tf.float64, name='k_mean_a',trainable=True)
#kernel_mean_l = tfp.util.TransformedVariable(1.0, tfb.Chain([tfb.Shift(np.float64(0.5)), tfb.Softplus()]),dtype=tf.float64, name='k_mean_l',trainable=True)

#kernel_len = tfp.math.psd_kernels.Constant(tfp.util.TransformedVariable(5.0, tfb.Softplus(),dtype=tf.float64, name='k_len',trainable=True), feature_ndims=1, validate_args=False, name='Constant_len_lk')
#kernel_amp = tfp.math.psd_kernels.Constant(tfp.util.TransformedVariable(5.0, tfb.Softplus(),dtype=tf.float64, name='k_amp',trainable=True), feature_ndims=1, validate_args=False, name='Constant_amp_lk')


#kernel_mean = tfk.ExpSinSquared(kernel_mean_a,kernel_mean_l,period=np.float64(24.0))

#import nsgp_vi_ns_mean first

#from nsgp_vi_ns_mean.py import nsgp_vi_ns_mean

#vgp = nsgp_vi_ns_mean(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  


In [ ]:
#vgp.trainable_variables

In [ ]:
#loss = vgp.optimize(BATCH_SIZE, SEG_LENGTH, NUM_EPOCHS=500)


In [ ]:
import pickle5 as pickle

#Load the inducing points and the optimized parameters 
with open('../results/opt_params_Sceaux_power_3nsgp.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
inducing_index_points = np.load('../results/T_ind_Sceaux_power_3nsgp.npy')    

In [ ]:
vgp = nsgp_vi_3ns_params(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  


In [ ]:
#Load the parameters !!!
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)

In [ ]:
ZZ = np.linspace(0,24,200)[:,None]

[mean_mean,len_mean,amp_mean], [mean_var,len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

mean_mean = mean_mean[0,:,0].numpy()
mean_std = mean_var[:,0].numpy()**0.5


len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5


f, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(30,10))

ax1.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)

ax1.set_xlabel('Time',size=15)
ax1.set_ylabel('Power consumption persistence',size=15)
ax1.text(-0.15,1,'A', size=25, transform=ax1.transAxes)
ax1.tick_params(axis='both', which='major', labelsize=15)
tick_locator = tick.MaxNLocator(nbins=7)

ax2.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax2.set_xlabel('Time',size=15)
ax2.set_ylabel('Power consumption variance',size=15)
ax2.text(-0.15,1,'B', size=25, transform=ax2.transAxes)
ax2.tick_params(axis='both', which='major', labelsize=15)
tick_locator = tick.MaxNLocator(nbins=7)

ax3.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax3.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax3.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax3.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)

ax3.set_xlabel('Time',size=15)
ax3.set_ylabel('Power consumption mean',size=15)
ax3.text(-0.15,1,'C', size=25, transform=ax3.transAxes)
ax3.tick_params(axis='both', which='major', labelsize=15)
tick_locator = tick.MaxNLocator(nbins=7)

plt.savefig("power_inference_3nsgp.pdf",dpi=600)

plt.show()


In [ ]:
# run the following boxes only for section 6.3 of the paper; calculates the log predictive density 

# Sceaux household
# take the last 200,000 elements
T_pred = T[-200000:]
X_pred = X[-200000:]

#Tetuan datasets; take the last 10,000 elements
#T_pred = T[-10000:]
#X_pred = X[-10000:]

num_prediction_points_ = len(T_pred)

In [ ]:
BATCH_SIZE=8
SEG_LENGTH=500


class segment_generator:
    def __iter__(self):
        
        # loop over segments
        self.j = 0
        self.max_j = num_prediction_points_//SEG_LENGTH
        
        
        return self

    def __next__(self):
        

        if self.j==self.max_j:
            raise StopIteration

        TT = T_pred[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]#seg_length points
        XX = X_pred[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
    
        self.j += 1

        return TT,XX

        
dataset = tf.data.Dataset.from_generator(segment_generator, (tf.float64)) 
dataset = dataset.map(lambda dd: (dd[0],dd[1]))
dataset = dataset.shuffle(500)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
#for d in dataset:
 #   print(d[0].shape,d[1].shape)

In [ ]:

#calculate the predictive log probability density
log_l = 0.0 
for d in dataset:
    x_pred_batch = d[0] # time
    y_pred_batch = d[1] # data
    kl_weight = tf.reduce_sum(tf.ones_like(x_pred_batch))/num_prediction_points_ 
    loss = vgp.variational_loss(observations=y_pred_batch,observation_index_points=x_pred_batch,kl_weight=kl_weight) 
    log_l = log_l + loss
log_l = - log_l
    

In [ ]:
log_l